In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
from lxml import etree
import re
import os

In [2]:
#欲查詢資料匯入
src = pd.read_excel(r"input/comp_info.xlsx")  #匯入統編檔案，欄位名稱為ID
p_key = src['ID']

df_raw = {}
df_raw2 = {}
for i in range(12):
    df_raw['A%s' %i] = []
for i in range(1,16):
    df_raw2['B%s' %i] = []
    df_raw2['C%s' %i] = []
    df_raw2['D%s' %i] = []
df_raw2['經理人']=[]  #經理人

In [3]:
# --- 此為產生董事名單、法人、股份，存至 B1-15,C1-15,D1-15 ---
def pageboard():

    for s in range(1,16):
        try:
            BB = selector.xpath(
                '//*[@id="tabShareHolderContent"]/div[3]/table/tbody/tr['+str(s)+']/td[3]/text()')[0].strip()
            df_raw2['B%s' %s].append(BB)
        except:
            df_raw2['B%s' %s].append('')
            df_raw2['C%s' %s].append('')
            df_raw2['D%s' %s].append('')
            continue
        else:
            try:
                CC= selector.xpath(
                    '//*[@id="tabShareHolderContent"]/div[3]/table/tbody/tr['+str(s)+']/td[5]/text()')[0]
                df_raw2['C%s' %s].append(CC)
            except:
                df_raw2['C%s' %s].append('')
            finally:
                try:
                    DD = selector.xpath(
                        '//*[@id="tabShareHolderContent"]/div[3]/table/tbody/tr['+str(s)+']/td[4]/a/text()')[0].strip()
                    df_raw2['D%s' %s].append(DD)   #此格為空會出錯
                except:
                    df_raw2['D%s' %s].append('')

# --- 此函數儲存經理人資料，存入經理人中 ---
def pagemanage():
    M=''
    try:
        M0 = selector.xpath(
            '//*[@id="tabMgrContent"]/div/table/tbody/tr/td[2]/text()')[0].strip()
        M += M0
    except:
        for i in range(25):
            try:
                M1 = selector.xpath(
                    '//*[@id="tabMgrContent"]/div/table/tbody/tr['+str(i)+'[/td[2]/text()')[0].strip()
                M+=','+M1
                print('b')
            except:
                break

    df_raw2['經理人'].append(M)

    
# --- 此函數為首頁內容下載，存至 A0-A11 ---
def page1(countN):
        #不同公司,商業司網頁欄位位置不同
    collist=['[\u7d71\u4e00\u7de8\u865f]{4}','[\u516c\u53f8\u72c0\u6cc1]{4}',
             '[\u80a1\u6b0a\u72c0\u6cc1]{4}','[\u516c\u53f8\u540d\u7a31]{4}',
             '[\u8cc7\u672c\u7e3d\u984d]{4}','[\u5be6\u6536\u8cc7\u672c\u984d]{5}',
             '[\u4ee3\u8868\u4eba\u59d3\u540d]{5}','[\u516c\u53f8\u6240\u5728\u5730]{5}',
             '[\u767b\u8a18\u6a5f\u95dc]{4}','[\u6838\u51c6\u8a2d\u7acb\u65e5\u671f]{6}',
             '[\u6700\u5f8c\u6838\u51c6\u8b8a\u66f4\u65e5\u671f]{8}']
    #以上定義標題名稱，並寫成正則形式
    it={'[\u7d71\u4e00\u7de8\u865f]{4}':df_raw['A0'],
        '[\u516c\u53f8\u72c0\u6cc1]{4}':df_raw['A1'],
        '[\u80a1\u6b0a\u72c0\u6cc1]{4}':df_raw['A2'],
        '[\u516c\u53f8\u540d\u7a31]{4}':df_raw['A3'],
        '[\u8cc7\u672c\u7e3d\u984d]{4}':df_raw['A4'],
        '[\u5be6\u6536\u8cc7\u672c\u984d]{5}':df_raw['A5'],
        '[\u4ee3\u8868\u4eba\u59d3\u540d]{5}':df_raw['A6'],
        '[\u516c\u53f8\u6240\u5728\u5730]{5}':df_raw['A7'],
        '[\u767b\u8a18\u6a5f\u95dc]{4}':df_raw['A8'],
        '[\u6838\u51c6\u8a2d\u7acb\u65e5\u671f]{6}':df_raw['A9'],
        '[\u6700\u5f8c\u6838\u51c6\u8b8a\u66f4\u65e5\u671f]{8}':df_raw['A10'],
        '[\u6240\u71df\u4e8b\u696d\u8cc7\u6599]{6}':df_raw['A11']}

    for i in range(1,20):
        try:
            colname=selector.xpath('//*[@id="tabCmpyContent"]/div/table/tbody/tr['+str(i)+']/td[1]/text()')[0].strip()
            
            for t in collist:
                if re.match(t,colname)!= None:        #判斷標題名稱是否符合正則           
                    cont=selector.xpath('//*[@id="tabCmpyContent"]/div/table/tbody/tr['+str(i)+']/td[2]/text()')[0].strip()
                    it[t].append(cont)
                    break
                elif re.match('[\u6240\u71df\u4e8b\u696d\u8cc7\u6599]{6}',colname)!=None:   #事業資料需要額外處理
                    busine=''
                    busi = selector.xpath('//*[@id="tabCmpyContent"]/div/table/tbody/tr['+str(i)+']/td[2]/text()')
                    for b in range(int(len(busi)/2)):
                        b_noempty=re.findall('\S+',busi[b])   #把空白篩掉
                        for c in b_noempty:
                            busine += c
                    it['[\u6240\u71df\u4e8b\u696d\u8cc7\u6599]{6}'].append(busine)
                    break       
                else:
                    continue
        
        except:
            continue
            
    for n in range(12):                                #為解決該公司無此欄位
        if len(df_raw['A%s' %n]) != countN:           #countN為公司數目
            df_raw['A%s' %n].append('')

In [4]:
#crawl完整

web = webdriver.Chrome(r"/Users/adam/Jupyter/chromedriver")
web.get('https://findbiz.nat.gov.tw/fts/query/QueryList/queryList.do')
time.sleep(2)

count=1   #計算第幾家公司
for idn in p_key:
    
    try:
        aa = web.find_element_by_xpath('//*[@id="qryCond"]')
        aa.clear()
        aa.send_keys(str(idn))    #輸入統編

        web.find_element_by_css_selector('#qryBtn').click()    #點擊查詢
        time.sleep(0.5)
        web.find_element_by_css_selector(
            '#vParagraph > div > div.panel-heading.companyName > a:nth-child(1)').click() #點詳細資訊
        time.sleep(1)

        selector = etree.HTML(web.page_source) #存下原始碼
        time.sleep(0.5)

        page1(count)   #基本資料
        pageboard()     #董事會
        pagemanage()    #經理人

        web.find_element_by_css_selector(
            '#bs-example-navbar-collapse-1 > ul > li:nth-child(1) > a').click()  #點擊重新查詢
        time.sleep(0.5)

        count += 1  #跳往下一家公司
    
    except:
        print(str(idn)+'查無資料')
        continue

web.close()

df_main = pd.DataFrame(df_raw)
df_board = pd.DataFrame(df_raw2)

df_main.rename(columns = {"A0":"統一編號","A1":"公司狀況","A2":"股權狀況",
                          "A3":"公司名稱","A4":"資本總額","A5":"實收資本額",
                          "A6":'代表人姓名',"A7":'公司所在地',"A8":'登記機關',
                          "A9":'核准設立日期',"A10":'最後核准變更日期',"A11":'所營事業資料'}
              , inplace = True)

df_board["統一編號"] = df_raw["A0"]

df_main.to_excel("Comp.xlsx")
df_board.to_excel("Comp_Board.xlsx")

73704126查無資料


In [5]:
pd.set_option("display.max_columns", None)
df_main

,統一編號,公司狀況,股權狀況,公司名稱,資本總額,實收資本額,代表人姓名,公司所在地,登記機關,核准設立日期,最後核准變更日期,所營事業資料
0,83504978,核准設立,,將來商業銀行股份有限公司,"10,000,000,000","10,000,000,000",鍾福貴,臺北市大安區敦化南路2段95號6、7、8、9樓,經濟部商業司,109年01月31日,109年03月31日,H101021商業銀行業H101021商業銀行業
1,83477590,核准設立,,連線商業銀行股份有限公司,"10,000,000,000","10,000,000,000",黃仁埈,臺北市信義區基隆路1段206號15樓,經濟部商業司,109年03月09日,,H101021商業銀行業H101021商業銀行業
2,83794245,核准設立,僑外資,樂天國際商業銀行股份有限公司,"10,000,000,000","10,000,000,000",簡明仁,臺北市中山區松江路433號11樓,經濟部商業司,109年05月18日,,H101021商業銀行業H101021商業銀行業
3,22099131,核准設立,僑外資,台灣積體電路製造股份有限公司,"280,500,000,000","259,303,804,580",Liu Te-Yin Mark(劉德音),新竹科學園區新竹市力行六路8號,科技部新竹科學園區管理局,076年02月21日,109年04月13日,CC01080電子零組件製造業CC01090電池製造業CC01040照明設備製造業IG030...
4,03522003,核准設立,,遠東航空股份有限公司,"8,000,000,000","3,324,000,000",張綱維,臺北市松山區精忠里敦化北路405巷123弄5號,經濟部商業司,048年10月09日,109年03月06日,F114070航空器及其零件批發業F214070航空器及其零件零售業F113010機械批發業...


In [6]:
df_board

,B1,C1,D1,B2,C2,D2,B3,C3,D3,B4,C4,D4,B5,C5,D5,B6,C6,D6,B7,C7,D7,B8,C8,D8,B9,C9,D9,B10,C10,D10,B11,C11,D11,B12,C12,D12,B13,C13,D13,B14,C14,D14,B15,C15,D15,經理人,統一編號
0,鍾福貴,0,,馬宏燦,0,,屈美惠,0,,林榮賜,"419,000,000",中華電信股份有限公司,吳麗秀,"419,000,000",中華電信股份有限公司,施勝耀,"419,000,000",中華電信股份有限公司,胡光華,"251,000,000",兆豐國際商業銀行股份有限公司,林瑞雲,"251,000,000",兆豐國際商業銀行股份有限公司,李靜怡,"251,000,000",兆豐國際商業銀行股份有限公司,陳碧天,"251,000,000",兆豐國際商業銀行股份有限公司,黃敏義,"100,000,000",新光人壽保險股份有限公司,劉奕成,"99,000,000",全聯實業股份有限公司,林紹華,"70,000,000",凱基商業銀行股份有限公司,許建隆,"21,000,000",關貿網路股份有限公司,洪國超,0,,劉奕成,83504978
1,黃仁埈,"499,000,000",台灣連線金融科技股份有限公司,黃以孟,0,,陳金榮,0,,Youngsu Ko,"499,000,000",台灣連線金融科技股份有限公司,Bora Choi,"499,000,000",台灣連線金融科技股份有限公司,温嘉仁,"499,000,000",台灣連線金融科技股份有限公司,蘇清偉,"251,000,000",台北富邦商業銀行股份有限公司,蔡瀛凱,"251,000,000",台北富邦商業銀行股份有限公司,黃鈞淇,"251,000,000",台北富邦商業銀行股份有限公司,詹英柏,"50,000,000",聯邦商業銀行股份有限公司,朱盈璇,"50,000,000",中國信託商業銀行股份有限公司,Alex Manson,"50,000,000",渣打國際商業銀行股份有限公司,林之晨,"50,000,000",台灣大哥大股份有限公司,尹德洋,"50,000,000",遠傳電信股份有限公司,陳立人,0,,,83477590
2,簡明仁,"490,000,000",國票金融控股股份有限公司,永井 啓之,0,,佐伯 和彥,0,,魏啓林,"490,000,000",國票金融控股股份有限公司,石井 英治,0,,王東和,0,,東林 知隆,0,,內山 洋之,0,,,,,,,,,,,,,,,,,,,,,,,,83794245
3,Liu Te-Yin Mark(劉德音),"12,913,114",,魏哲家,"7,179,207",,陳美伶,"1,653,709,980",,曾繁城,"34,472,675",,彼得‧邦菲爵士,0,,施振榮,"1,480,286",,陳國慈,0,,麥克‧史賓林特,0,,摩西‧蓋弗瑞洛夫,0,,,,,,,,,,,,,,,,,,,,魏哲家,22099131
4,張綱維,"2,954",樺壹租賃股份有限公司,林育陞,"2,954",樺壹租賃股份有限公司,缺額,,,缺額,,,許慧娟,20,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,張綱維,03522003
